In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.data import (
    load_loan_payments_dataset_scoring, 
    load_loan_agencies, 
    load_loan_paymets,
    load_loan_funding_info,
    load_dindex_dataset
)
from pathlib import Path

DATA_DIR = Path("../data").resolve()

### Load data

In [2]:
prediction_samples = load_loan_payments_dataset_scoring(
    filepath=DATA_DIR / "loan_payments_dataset_scoring.csv"
)

payments = load_loan_paymets(
    filepath=DATA_DIR / "loan_payments_dataset.csv"
)

funding = load_loan_funding_info(
    filepath=DATA_DIR / "loan_funding_origination_info.csv"
)

dindex = load_dindex_dataset(
    filepath=DATA_DIR / "loan_dindexedto_dataset.csv"
)

agencies = load_loan_agencies(
    filepath=DATA_DIR / "loan_agency_product_name.csv",
    just_naboo=True
)

agencies = agencies[agencies.AgencyId.isin(funding.AgencyId)]

### Payments

In [3]:
payments = payments.drop(
    labels=["PaymentId", "PaymentAmount", "PaymentInterest", "PaymentVAT", "PaymentDueDate", "PaymentTransferDate"], 
    axis=1)

payments.head()

,LoanId,PaymentSource,PaymentType,PaymentTypeDescription,PaymentPrincipal,PaymentProcessingDate
0,303,Contractor,NaN,NaN,0.53,2020-01-06
1,1326,Contractor,NaN,NaN,20.00,2020-01-06
2,1763,Contractor,NaN,NaN,500.00,2020-01-06
3,1829,Contractor,NaN,NaN,399.00,2020-03-13
4,1884,Contractor,NaN,NaN,465.00,2020-01-06


In [4]:
payments = payments.sample(100000)

### Preprocessing Base

In [5]:
def base_preprocessing(payments: pd.DataFrame) -> pd.DataFrame:
    """Base preprocessing fn that performs the following tasks:
    
    - Merge with funding
    - Merge with agencies
    - Merge with dindex
    - Sorts the df by LoanId, PaymentProcessingDate and PaymentId

    Arguments:
        payments: Payments dataframe to preprocess
    """
    payments = pd.merge(left=payments, right=funding, on="LoanId")
    payments = pd.merge(left=payments, right=agencies, on="AgencyId")
    payments = pd.merge(left=payments, right=dindex, on="LoanId", suffixes=("_agency", "_dinx"))

    payments = payments.sort_values(by=["LoanId", "PaymentProcessingDate"])

    return payments

### Preprocessing 1

**Funding**

- Calcula Closing Date
- Saca aquellos loans sin periodicidad (cancelados)


**Payments**

- Saca PaymentID
- Saca PaymentInterest
- Saca PaymentVAT
- Saca PaymentDueDate (otra opción: inferirla en base a otros pagos del mismo crédito)
- Saca PaymentTransferDate (otra opción: inferirla en base a otros pagos del mismo crédito)

In [6]:
from datetime import timedelta, datetime
def calculate_closing_date(row: pd.Series) -> datetime:
    """Calculates the loan's closing date based on origination date, terms and periodicity"""
    delta = None

    if row.Periodicity == "Biweekly":
        delta = 14
    elif row.Periodicity == "Monthly":
        delta = 30
    elif row.Periodicity == "Weekly":
        delta = 7
    else:
        raise Exception(f"Unmapped periodicity! {row.Periodicity}")
    
    return row.OriginationDate + timedelta(days=row.Term * delta)


def preprocessing_v1(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocessing function that performs the following tasks:
    
    - Removes entries with Periodicity NaN (cancelled loans)
    - Calculates if the payment is in default or not
    """
    df = base_preprocessing(df)

    df = df[~df.Periodicity.isna()]

    df["ClosingDate"] = df.apply(calculate_closing_date, axis=1)

    df["InDefault"] = df["PaymentProcessingDate"] > df["ClosingDate"]

    df.drop(labels=["Country", "ProductName_dinx"], axis=1, inplace=True)
    df.fillna(0, inplace=True)

    return df

In [7]:
prediction_samples = preprocessing_v1(prediction_samples)

prediction_samples.head()

,LoanId,PaymentSource,PaymentType,PaymentTypeDescription,PaymentPrincipal,PaymentProcessingDate,PaymentCode,AgencyId,FundingID,OriginationDate,InstallmentAmount,Periodicity,LoanOperationalStatus,VAT,Term,ProductName_agency,Type,D-IndexedTo,ClosingDate,InDefault
1720024,3681,Alternative,0.0,0,0.0,2020-08-10,012008103681,930,6,2012-06-26,25489.5,Biweekly,Paid-off,11,36,PDL,Base,Hopp,2013-11-12,True
1720025,3681,Alternative,0.0,0,0.0,2020-08-11,012008113681,930,6,2012-06-26,25489.5,Biweekly,Paid-off,11,36,PDL,Base,Hopp,2013-11-12,True
1718814,3846,Regular,0.0,0,0.0,2020-07-22,002007223846,930,6,2012-09-14,200536.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-20,True
1718807,3846,Regular,0.0,0,0.0,2020-08-17,002008173846,930,6,2012-09-14,200536.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-20,True
1718806,3846,Regular,0.0,0,0.0,2020-08-20,002008203846,930,6,2012-09-14,200536.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-20,True


In [8]:
payments = preprocessing_v1(payments)

payments.head()

,LoanId,PaymentSource,PaymentType,PaymentTypeDescription,PaymentPrincipal,PaymentProcessingDate,AgencyId,FundingID,OriginationDate,InstallmentAmount,Periodicity,LoanOperationalStatus,VAT,Term,ProductName_agency,Type,D-IndexedTo,ClosingDate,InDefault
60612,3811,Regular,0.0,0,164.99,2019-11-21,930,384,2012-09-07,28648.0,Biweekly,Paid-off,11,96,PDL,Base,Hopp,2016-05-13,True
60443,3846,Regular,0.0,0,0.00,2019-12-06,930,6,2012-09-14,200536.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-20,True
60585,3880,Regular,0.0,0,0.00,2020-02-10,930,6,2012-09-20,143240.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-26,True
60469,3881,Regular,0.0,0,0.00,2019-04-23,930,6,2012-09-20,229184.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-26,True
60468,3881,Regular,0.0,0,0.00,2019-08-06,930,6,2012-09-20,229184.0,Biweekly,Debtor,11,96,PDL,Base,Hopp,2016-05-26,True


### Encoding: OneHotEncoding con get_dummies

In [9]:
payments = pd.get_dummies(payments)

### Train

In [10]:
from datetime import datetime

X = payments.drop("PaymentPrincipal", axis=1)
y = payments.PaymentPrincipal

split_date = datetime(2020, 3, 31)

df_train = payments[payments.PaymentProcessingDate <= split_date]
df_test = payments[payments.PaymentProcessingDate > split_date]

X_train = df_train.drop(["PaymentPrincipal", "OriginationDate", "ClosingDate", "PaymentProcessingDate"], axis=1)
X_test  = df_test.drop(["PaymentPrincipal", "OriginationDate", "ClosingDate", "PaymentProcessingDate"], axis=1)

y_train = df_train.PaymentPrincipal
y_test = df_test.PaymentPrincipal

print(f"""
- X_train: {len(X_train)} records
- X_test: {len(X_test)} records
- y_train: {len(y_train)} records
- y_test: {len(y_test)} records
"""
)


- X_train: 84770 records
- X_test: 15230 records
- y_train: 84770 records
- y_test: 15230 records



In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# create a Random Forest classifier
linear_model = LinearRegression()

# fit it on the training set
linear_model.fit(X_train, y_train)

# make predictions on the test set
y_pred = linear_model.predict(X_test)

# calculate root mean squared error
mae = mean_absolute_error(y_test, y_pred)

print("Linear model MAE:", mae)

Linear model MAE: 436.5883844311156


### Predictions

In [ ]:
prediction_samples.drop(labels=["OriginationDate", "ClosingDate", "PaymentProcessingDate"], axis=1, inplace=True)

prediction_samples = pd.get_dummies(prediction_samples)

In [ ]:
prediction_samples.PaymentPrincipal = linear_model.predict(prediction_samples)